In [166]:
import pandas as pd
import os

In [167]:
path = "/Users/danielcm/Desktop/Sycuro/Projects/Diabetes/t1d_db_fixed_discussed/"
os.chdir(path)

In [216]:
#This section makes a new file (and df) for every barrnap blast output with the necessary headers
master_file = pd.read_csv("../WGS_all_runs/FINAL_report_WGS021224.csv")
subset_master_file = master_file[['Index','Sample ID','Consortia','Selected for Downstream',
                    'Sanger classification (v3v4) Source','GTDBtk Species Classification (pplacer)']]
input_file_path = path+"barrnap_blast/"
for file in sorted(os.listdir(input_file_path)):
    blast_file = pd.read_csv(input_file_path+file,header=None)
    blast_file.columns = ["Sequence ID","Matched ID","Bit score","Score","E-value","% of identity","Query coverage HSP"]
    number_of_16S = len(blast_file['Sequence ID'].unique())
    blast_file['Number of 16S found'] = number_of_16S
    filtered_blast_file = blast_file[(blast_file['% of identity']>=98.6) & (blast_file['E-value']<=1e-100)].copy() #Filtering by e-value and identity
    blast_species = filtered_blast_file['Matched ID'].tolist()
    matched_species = []
    for item in blast_species:
        split = item.split(" ",1)
        species_name = split[-1]
        matched_species.append(species_name)
    filtered_blast_file['Matched species'] = matched_species #All of them that were found at high quality
    tmp_id_name = "_".join(file.split("_")[3:7]).split(".",1)[0] #This assigns just the sample ID without the .csv or the 'prokka_blast' part
    filtered_blast_file["Sample ID"] = tmp_id_name
    unique_species = filtered_blast_file["Matched species"].unique() #This makes an array (or a list?) of unique species found by BLASTn
    genus_list = []
    for item in blast_species:
        genus = item.split(" ")[1]
        genus_list.append(genus)
    #print(genus_list)
    filtered_blast_file['Matched genus'] = genus_list
    unique_genus = filtered_blast_file["Matched genus"].unique() #This makes a list of unique genus found by BLASTn
    unique_species = list(unique_species)
    unique_genus = list(unique_genus)
    filtered_blast_file['Unique species'] = [unique_species]*len(filtered_blast_file)
    filtered_blast_file['Unique genus'] = [unique_genus]*len(filtered_blast_file)
    filtered_blast_file['No. of unique species'] = len(unique_species)
    filtered_blast_file['No. of unique genera'] = len(unique_genus)
    filtered_blast_file = filtered_blast_file[['Sample ID','Sequence ID','Matched ID','Bit score','Score','E-value','% of identity','Query coverage HSP','Number of 16S found',
                          'Matched species','No. of unique species','Unique species','Matched genus','No. of unique genera','Unique genus']]
    
    filtered_blast_file.to_csv(tmp_id_name+".csv",sep=',')